# OData Queries with Digital Thread Services

This notebook demonstrates how to use OData queries with the Digital Thread Services data model. It covers the schema structure and various OData query capabilities including filtering, expansion, ordering, and aggregation.

## Prerequisites

- Digital Thread Services (DTS) running and accessible
- Python environment with the datastore client library
- Sample test data in the datastore

## Table of Contents

1. [Schema Overview](#schema-overview)
2. [Basic Setup and Connection](#basic-setup-and-connection)
3. [Entity Relationships](#entity-relationships)
4. [Basic Queries](#basic-queries)
5. [Filtering Data](#filtering-data)
6. [Expanding Related Data](#expanding-related-data)
7. [Ordering and Paging](#ordering-and-paging)
8. [Aggregation and Grouping](#aggregation-and-grouping)
9. [Working with Measurements and Values](#working-with-measurements-and-values)
10. [Advanced Query Patterns](#advanced-query-patterns)
11. [Performance Tips](#performance-tips)

## Schema Overview

The Digital Thread Services (DTS) data model is designed to capture comprehensive test execution data. The schema consists of several key entity types that work together to represent the complete testing lifecycle.

### Core Entities

**TestResult**: The top-level entity representing a complete test execution
- Contains metadata about the test run
- Links to UUT (Unit Under Test) information
- References test station, operator, and test description

**Step**: Individual test steps within a test result
- Hierarchical structure supporting nested test steps
- Contains step-specific metadata and custom extensions
- Links to measurements and conditions collected during the step

**Measurement**: Actual measurement data collected during testing
- Supports various data types (numeric, waveform, etc.)
- Includes timing information and outcome status
- Can be associated with steps or directly with test results

**Condition**: Test conditions and parameters
- Environmental conditions, test parameters, and setup information
- Associated with steps or test results

### Metadata Entities

**UUT (Unit Under Test)**: Product information being tested
- Part numbers, specifications, and product metadata
- Links to specific UUT instances

**UutInstance**: Specific instance of a UUT
- Serial numbers, manufacturing dates
- Traceability information for individual units

**Hardware/Software Items**: Test equipment and software used
- Instrument information, calibration data
- Software versions and configurations

**Test Station, Operator, Test Description**: Context entities
- Test environment and personnel information
- Test procedure and documentation references

### Navigation and Relationships

The schema uses navigation properties to establish relationships between entities:
- **One-to-Many**: TestResult → Steps → Measurements
- **Many-to-Many**: Measurements ↔ HardwareItems (via navigation properties)
- **Reference Lookups**: Measurements → TestStation, Operator, etc.

### Extensions and Custom Metadata

Most entities support an `Extensions` property of type `DataStore.Extensions`:
- Open type allowing custom properties
- Enables domain-specific metadata without schema changes
- Supports flexible data models for different test domains

## Basic Setup and Connection

This section will demonstrate how to establish a connection to the Digital Thread Services and set up the OData client for querying.

## Entity Relationships

Understanding the relationships between entities is crucial for effective OData queries. This section explores how entities are connected and how to navigate these relationships.

### Primary Relationships

**TestResult as the Central Hub**
- Most entities can be reached from TestResult
- TestResult → Steps (one-to-many)
- TestResult → Measurements (one-to-many, direct and via Steps)
- TestResult → UutInstance → Uut (many-to-one relationships)

**Measurement Relationships**
- Measurements can exist at TestResult level or Step level
- Each measurement can reference multiple HardwareItems and SoftwareItems
- Measurements link to contextual entities (Operator, TestStation, etc.)

**Hierarchical Data**
- Steps can contain nested steps via navigation properties
- Path entities provide hierarchical organization
- Custom extensions enable domain-specific hierarchies

### Navigation Property Patterns

The schema uses several navigation patterns:
- **Direct References**: Foreign key relationships with navigation properties
- **Collection Navigation**: One-to-many relationships (e.g., TestResult.Measurements)
- **Bidirectional Navigation**: Many-to-many relationships through collections

## Basic Queries

This section demonstrates fundamental OData query operations including:
- Retrieving entity collections
- Getting individual entities by ID
- Basic projection (selecting specific properties)
- Understanding OData response formats

### Entity Collections
Examples of querying the main entity sets:
- TestResults
- Measurements  
- Steps
- UutInstances

### Individual Entity Retrieval
How to get specific entities using their primary keys and understand the response structure.

## Filtering Data

OData provides powerful filtering capabilities using the `$filter` query parameter. This section covers:

### Basic Filtering Operations
- **Comparison Operators**: eq, ne, gt, ge, lt, le
- **Logical Operators**: and, or, not
- **String Functions**: contains, startswith, endswith, length, tolower, toupper

### Property-Based Filtering
Examples filtering on common properties:
- TestResult Name filtering
- Measurement DataType filtering  
- UutInstance SerialNumber filtering
- Date/time range filtering using StartTime and EndTime

### Filtering on Enums
Working with enumeration values:
- Measurement Outcome filtering (Passed, Failed, Indeterminate)
- Alias TargetType filtering

### Complex Filtering
- Combining multiple conditions
- Filtering on navigation properties
- Using parentheses for complex logic
- Null value handling

### Date and Time Filtering
- Working with DateTimeOffset properties
- Time range queries
- Relative date filtering

## Expanding Related Data

The `$expand` query parameter allows you to include related entity data in a single request, reducing the number of round trips needed.

### Basic Expansion
- Expanding single navigation properties
- Understanding the expanded response structure
- Performance considerations for expansion

### Multi-Level Expansion
- Expanding nested relationships (e.g., TestResult → Steps → Measurements)
- Limiting expansion depth
- Selective property expansion

### Expansion with Filtering
- Combining `$expand` with `$filter` on related entities
- Filtering the expanded data
- Complex expansion scenarios

### Common Expansion Patterns
Examples of typical expansion scenarios:
- **TestResult with Steps and Measurements**: Complete test execution data
- **Measurement with Hardware/Software Items**: Understanding test setup
- **UutInstance with Uut Details**: Product and instance information
- **TestResult with Operator and TestStation**: Test context information

### Expansion Performance
- Best practices for efficient expansion
- When to use expansion vs. separate queries
- Limiting expanded data with `$select`

## Ordering and Paging

This section covers sorting results and handling large datasets through paging mechanisms.

### Ordering Results
Using the `$orderby` query parameter:
- **Single Property Ordering**: Ascending and descending sorts
- **Multiple Property Ordering**: Primary and secondary sort criteria
- **Ordering by Related Properties**: Sorting by navigation property values

### Common Ordering Scenarios
- TestResults by creation time or completion time
- Measurements by ParametricIndex or StartTime
- UutInstances by SerialNumber or ManufactureDate
- Steps by hierarchical order

### Paging Large Result Sets
Managing large datasets efficiently:
- **$top**: Limiting the number of results returned
- **$skip**: Skipping records for pagination
- **@odata.nextLink**: Server-driven paging
- **$count**: Getting total result counts

### Paging Strategies
- **Client-Driven Paging**: Using $top and $skip
- **Server-Driven Paging**: Handling continuation tokens
- **Combining with Filtering**: Efficient filtered paging
- **Performance Considerations**: Optimal page sizes

### Advanced Ordering
- Ordering by complex expressions
- Ordering by computed values
- Ordering by extension properties

## Aggregation and Grouping

This section explores data aggregation capabilities for analytical queries and reporting.

### Basic Aggregation
Using `$apply` for data aggregation:
- **Count**: Counting entities and groups
- **Sum, Average, Min, Max**: Numeric aggregations
- **Group By**: Grouping data by properties

### Grouping Patterns
Common grouping scenarios:
- **Test Results by Outcome**: Pass/fail statistics
- **Measurements by DataType**: Understanding data distribution  
- **Test Results by TestStation**: Station utilization analysis
- **Measurements by Time Periods**: Temporal analysis

### Complex Aggregations
- **Multi-level Grouping**: Nested grouping operations
- **Conditional Aggregations**: Aggregating with filters
- **Computed Aggregations**: Custom calculations
- **Aggregating Related Data**: Cross-entity aggregations

### Analytical Queries
Examples of business intelligence queries:
- Test yield analysis by product and time period
- Equipment utilization reports
- Operator performance metrics
- Failure analysis by test step

### Performance Considerations
- Indexing strategies for aggregation queries
- When to use aggregation vs. client-side processing
- Memory and performance implications

## Working with Measurements and Values

Measurements are central to the Digital Thread Services schema. This section focuses on querying measurement data and retrieving actual values.

### Measurement Types and Data Types
Understanding different measurement data types:
- **Scalar Values**: Single numeric measurements
- **Waveform Data**: Time-series data with T0, DT, and YData
- **XY Data**: Paired X-Y coordinate data
- **Complex Data**: Custom data structures

### Retrieving Measurement Values
Using bound functions to get measurement data:
- **GetValue()**: Retrieving individual measurement values
- **GetValues()**: Batch retrieval of multiple measurement values
- **GetWaveform()**: Specialized waveform data retrieval
- **GetWaveforms()**: Batch waveform retrieval

### Working with Monikers
Understanding data location and retrieval:
- **GetMoniker()**: Getting data location information
- **GetMonikers()**: Batch moniker retrieval
- **Understanding Moniker Properties**: DataSource, Location, DataInstance

### Measurement Filtering Strategies
Efficient queries for measurement data:
- Filtering by DataType for specific measurement types
- Time-based filtering using StartTime and EndTime
- Outcome-based filtering for pass/fail analysis
- Parametric index filtering for organized data

### Value Processing Patterns
Common patterns for working with measurement values:
- Handling different value types (WaveformValue, DoubleXYValue, etc.)
- Processing time-series data
- Working with units and labels
- Error handling for missing or invalid data

### Performance Optimization
- Selective measurement retrieval
- Batch operations for multiple measurements
- Caching strategies for frequently accessed data

## Advanced Query Patterns

This section covers sophisticated querying techniques and patterns for complex scenarios.

### Working with Extensions
Querying custom metadata in Extensions properties:
- **Extension Property Access**: Querying dynamic properties
- **Type-Safe Extension Queries**: Working with known extension schemas
- **Complex Extension Filtering**: Advanced filtering on extension data
- **Extension Property Projection**: Selecting specific extension properties

### Alias System Queries
Understanding and using the alias system:
- **Alias Resolution**: Finding entities by alias names
- **Multi-Target Aliases**: Working with different alias target types
- **Alias-Based Navigation**: Using aliases in complex queries
- **Alias Management**: Creating and maintaining alias mappings

### Path-Based Queries
Working with hierarchical path data:
- **Path Navigation**: Querying by path structures
- **Hierarchical Filtering**: Finding entities within path hierarchies
- **Path-Based Aggregation**: Rollup queries across path levels
- **Path Pattern Matching**: Finding entities matching path patterns

### Schema-Aware Queries
Leveraging schema information:
- **Schema Validation**: Ensuring data conforms to registered schemas
- **Schema-Based Filtering**: Querying by schema ID
- **Cross-Schema Queries**: Working with multiple schema versions
- **Schema Evolution**: Handling schema changes over time

### Complex Join Patterns
Advanced relationship queries:
- **Multi-Entity Joins**: Querying across multiple entity types
- **Conditional Joins**: Joins with complex conditions
- **Performance-Optimized Joins**: Efficient multi-table queries
- **Temporal Joins**: Time-based relationship queries

### Error Handling and Edge Cases
Robust query patterns:
- **Null Value Handling**: Dealing with optional properties
- **Error Information Processing**: Working with ErrorInformation complex types
- **Partial Data Scenarios**: Handling incomplete data sets
- **Query Timeout Handling**: Managing long-running queries

## Performance Tips

This final section provides guidance on optimizing OData queries for the Digital Thread Services schema.

### Query Optimization Strategies
- **Selective Projection**: Using `$select` to limit returned properties
- **Efficient Filtering**: Indexable properties and filter optimization
- **Expansion vs. Separate Queries**: When to expand and when to use multiple requests
- **Batch Operations**: Combining multiple operations efficiently

### Schema-Specific Optimizations
Best practices for the DTS schema:
- **Measurement Data Access**: Optimizing large measurement datasets
- **Hierarchical Data Queries**: Efficient step and path navigation
- **Time-Based Queries**: Leveraging temporal indexes
- **Extension Property Queries**: Optimizing dynamic property access

### Caching and Data Management
- **Client-Side Caching**: Caching frequently accessed reference data
- **Query Result Caching**: Avoiding redundant server requests
- **Incremental Data Loading**: Loading data progressively
- **Memory Management**: Handling large result sets efficiently

### Monitoring and Debugging
- **Query Performance Analysis**: Identifying slow queries
- **Server Load Considerations**: Balancing query complexity and frequency
- **Error Handling**: Graceful degradation and retry strategies
- **Logging and Diagnostics**: Troubleshooting query issues

### Best Practices Summary
- Start with simple queries and add complexity incrementally
- Use appropriate paging for large datasets
- Leverage server-side filtering and projection
- Consider the impact of expansion on performance
- Monitor query performance and adjust as needed
- Use batch operations for multiple related queries

---

## Next Steps

After completing this notebook, you should have a comprehensive understanding of:
- The Digital Thread Services data model structure
- OData query capabilities and syntax
- Performance optimization techniques
- Common query patterns for test data analysis

This knowledge will enable you to build effective applications and analytics on top of the Digital Thread Services platform.